In [1]:
import pandas as pd
import json
import timeit
import time

In [2]:
drone_last_action = time.time()
time.sleep(1.2) # needed for first action to be taken into account
drone_status = 'grounded'

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
cols_x = ['leftShoulder_x',
          'rightShoulder_x',
          'leftWrist_x',
          'rightWrist_x',
          'leftHip_x',
          'rightHip_x',
          'leftElbow_x',
          'rightElbow_x']

cols_y = [col.replace('x', 'y') for col in cols_x]

In [5]:
use_interpolation = True

In [6]:
if use_interpolation:
    start_time = timeit.default_timer()
    ms_since_start = timeit.default_timer()
    cols = sorted(cols_x + cols_y + ['ms_since_start'])
else:
    cols = sorted(cols_x + cols_y)

In [7]:
pose_json = '{"poses":[{"score":0.3305602508857298,"keypoints":[{"score":0.9985909461975098,"part":"nose","position":{"x":248.6576838258837,"y":256.0402913411458}},{"score":0.9997169375419617,"part":"leftEye","position":{"x":288.0274458087859,"y":236.07972819010416}},{"score":0.9891241788864136,"part":"rightEye","position":{"x":218.41943759605533,"y":244.81049397786458}},{"score":0.9666750431060791,"part":"leftEar","position":{"x":335.9838671137075,"y":290.9280598958333}},{"score":0.786156952381134,"part":"rightEar","position":{"x":183.74859819255892,"y":301.9474772135417}},{"score":0.657204270362854,"part":"leftShoulder","position":{"x":443.04252449410865,"y":472.5755121866862}},{"score":0.16853265464305878,"part":"rightShoulder","position":{"x":119.14107726050206,"y":453.5830586751302}},{"score":0.016177650541067123,"part":"leftElbow","position":{"x":493.0340886350538,"y":489.2761617024739}},{"score":0.008160887286067009,"part":"rightElbow","position":{"x":-15.11167832671619,"y":497.9180094401042}},{"score":0.004399956203997135,"part":"leftWrist","position":{"x":466.03887689309045,"y":504.0832194010417}},{"score":0.004726673010736704,"part":"rightWrist","position":{"x":43.51104736328125,"y":498.2728698730469}},{"score":0.004330772906541824,"part":"leftHip","position":{"x":397.88377405385506,"y":502.5487772623698}},{"score":0.0069005475379526615,"part":"rightHip","position":{"x":91.98278208247952,"y":488.4844584147135}},{"score":0.003729963442310691,"part":"leftKnee","position":{"x":444.6972150958953,"y":471.8383005777995}},{"score":0.0024850412737578154,"part":"rightKnee","position":{"x":7.370285284323771,"y":500.6000691731771}},{"score":0.0016965661197900772,"part":"leftAnkle","position":{"x":444.20433669793806,"y":484.8676361083984}},{"score":0.0009152236161753535,"part":"rightAnkle","position":{"x":25.47802734375,"y":479.93550135294595}}]}],"image":{"width":640,"height":480}}'

In [8]:
pose_empty_json = '{"poses": [], "image": {"width": 640, "height": 480}}'

In [9]:
df = pd.DataFrame(columns=cols)

In [10]:
def json_to_dict(pose_json):

    x = json.loads(pose_json)
    pose_dict = {}
    
    if len(x['poses']) == 1:
        for i in range(8):
            pose_dict[x['poses'][0]['keypoints'][i+5]['part'] +
                      '_x'] = x['poses'][0]['keypoints'][i + 5]['position']['x']
            pose_dict[x['poses'][0]['keypoints'][i+5]['part'] +
                      '_y'] = x['poses'][0]['keypoints'][i + 5]['position']['y']

    # del pose_dict['leftElbow_x']
    # del pose_dict['leftElbow_y']
    # del pose_dict['rightElbow_x']
    # del pose_dict['rightElbow_y']

    return pose_dict

In [11]:
def predict_movement(pose):
    
    movement = 5
    
    leftArm_x = pose['leftWrist_x'] - pose['leftShoulder_x']
    rightArm_x = pose['rightShoulder_x'] - pose['rightWrist_x']
    leftArm_y = pose['leftShoulder_y'] - pose['leftWrist_y']
    rightArm_y = pose['rightShoulder_y'] - pose['rightWrist_y']
    
    if leftArm_x > 60:
        movement = 1

    if rightArm_x > 60:
        movement = 2

    if ((leftArm_x > 60) & (rightArm_x > 60)):
        movement = 0

    if ((leftArm_y > 100) & (rightArm_y > 100)):
        movement = 3

    if ((leftArm_y < -100) & (rightArm_y < -100)):
        movement = 4
        
    return movement

In [12]:
def steer_drone(movement):
    global drone_last_action
    global drone_status
    if (time.time() - drone_last_action) > 1:
        #print(movements[movement] + ' | ' + drone_status)
        drone_last_action = time.time()
        if ((movement == 4) & (drone_status == 'grounded')):
            drone_status = 'flying'
            print('drone.takeoff()')
        if (drone_status != 'grounded'):
            if movement == 6:
                drone_status = 'grounded'
                print('drone.land()')
            if movement == 0:
                print("drone.flip('r')")
            if movement == 1:
                print('drone.rotate_ccw(45)')
            if movement == 2:
                print('drone.rotate_cw(45)')
            if movement == 5:
                print('drone.move_forward(1)')

In [13]:
predict_movement(json_to_dict(pose_json))

2

In [14]:
start = timeit.default_timer()
steer_drone(predict_movement(json_to_dict(pose_json)))
stop = timeit.default_timer()
print('Time: ', 1000*(stop - start))

Time:  0.3658465064457772


In [26]:
if use_interpolation:
    pose_new = pd.DataFrame(json_to_dict(pose_json), index=[0])
    pose_new.at[0,'ms_since_start'] = round(1000*(timeit.default_timer()-start_time))
    df = df.append(pose_new)
else:
    df = df.append(pd.DataFrame(json_to_dict(pose_json), index=[0]))
    df.to_csv('dataframe.csv',  index=False)

In [27]:
df.reset_index(inplace=True, drop=True)

In [28]:
df

,leftElbow_x,leftElbow_y,leftHip_x,leftHip_y,leftShoulder_x,leftShoulder_y,leftWrist_x,leftWrist_y,ms_since_start,rightElbow_x,rightElbow_y,rightHip_x,rightHip_y,rightShoulder_x,rightShoulder_y,rightWrist_x,rightWrist_y
0,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,6464.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
1,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,9231.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
2,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,9721.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
3,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,9905.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
4,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,10333.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
5,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,10744.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
6,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,10939.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
7,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,11098.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
8,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,11276.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
9,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,11945.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287


In [35]:
cut_off = df.iloc[df['ms_since_start'].argmax()]['ms_since_start'] - 2000

In [40]:
df_keep = df.loc[df['ms_since_start']>=cut_off]

In [41]:
df_discarded = df.loc[df['ms_since_start']<cut_off]
df_discarded.tail(3)

,leftElbow_x,leftElbow_y,leftHip_x,leftHip_y,leftShoulder_x,leftShoulder_y,leftWrist_x,leftWrist_y,ms_since_start,rightElbow_x,rightElbow_y,rightHip_x,rightHip_y,rightShoulder_x,rightShoulder_y,rightWrist_x,rightWrist_y
2,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,9721.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
3,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,9905.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
4,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,10333.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287


In [42]:
df_keep

,leftElbow_x,leftElbow_y,leftHip_x,leftHip_y,leftShoulder_x,leftShoulder_y,leftWrist_x,leftWrist_y,ms_since_start,rightElbow_x,rightElbow_y,rightHip_x,rightHip_y,rightShoulder_x,rightShoulder_y,rightWrist_x,rightWrist_y
5,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,10744.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
6,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,10939.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
7,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,11098.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
8,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,11276.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
9,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,11945.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
10,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,12359.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
11,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,12548.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287


In [47]:
df = pd.concat([df_discarded.tail(3), df_keep]).reset_index(drop=True)

In [48]:
df

,leftElbow_x,leftElbow_y,leftHip_x,leftHip_y,leftShoulder_x,leftShoulder_y,leftWrist_x,leftWrist_y,ms_since_start,rightElbow_x,rightElbow_y,rightHip_x,rightHip_y,rightShoulder_x,rightShoulder_y,rightWrist_x,rightWrist_y
0,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,9721.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
1,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,9905.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
2,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,10333.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
3,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,10744.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
4,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,10939.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
5,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,11098.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
6,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,11276.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
7,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,11945.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
8,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,12359.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
9,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,12548.0,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287


In [ ]:
for i in range(16):
    df = df.append(pd.DataFrame(json_to_dict(pose_json), index=[0]))

In [ ]:
len(df)

In [ ]:
pose_np = df.values.reshape(1, 17, 16)

In [ ]:
pose_np

In [ ]:
import os
os.getcwd()

In [ ]:
os.chdir('../..')

In [ ]:
from app_local.module import GestureTransformer
from sklearn.pipeline import make_pipeline

In [ ]:
processing_pipeline = make_pipeline(GestureTransformer(cols))

In [ ]:
processing_pipeline.fit_transform(pose_np)

In [ ]:
import numpy as np

In [ ]:
from keras.models import load_model
import h5py

In [ ]:
model = load_model('model_gesture.h5')

In [ ]:
movement = np.argmax(model.predict(pose_np)[0])

In [ ]:
movement